In [ ]:
import os
import openai
import pandas as pd
import tiktoken
import json
import requests
import pickle as pkl
from collections import defaultdict
from googlesearch import search
from fake_useragent import UserAgent
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import re
from time import sleep
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader, Dataset


In [ ]:
with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

In [ ]:
print(deployment_name)

In [ ]:
#To check if it's working

import os
import openai
openai.api_type = "azure"
openai.api_version = "2023-05-15" 

print(deployment_name)

response = openai.ChatCompletion.create(
    engine=deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    messages=[
        {"role": "system", "content": "Assistant is a large language model trained by OpenAI."},
        {"role": "user", "content": "Who were the founders of Microsoft?"}
    ]
)

print(response)

print(response['choices'][0]['message']['content'])

In [ ]:
system_message={"role": "system", "content": "You are a helpful assistant."}

In [ ]:
max_response_tokens = 500
token_limit = 4096
conversation = []
conversation.append(system_message)

In [ ]:
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        # print(message)
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [ ]:
def prompt_CNN(article):

    prompt="""

For the following article: {} 

Return a summary comprising of 3 sentences. With each sentence in a dash bulleted format.

""".format(article)

    return prompt

In [ ]:
def prompt(article):

    prompt="""

For the following article: {} 

Return a summary comprising of 1 sentence. With the sentence in a dash bulleted format.

""".format(article)

    return prompt

In [ ]:
def prompt_output_parse(output):
    first_split=output.split('\n')
    sentences=[]
    for items in first_split:
        sentences.append(items.split('-')[-1].strip())

    return sentences

In [ ]:
def deployment_stuff():
    with open('azure-configuration.json') as inputfile:
        azureconfig = json.load(inputfile)
    openai.api_key = azureconfig['key'] 
    openai.api_base = azureconfig['endpoint'] 
    openai.api_type = 'azure'
    openai.api_version = '2023-05-15' # this may change in the future

    deployment_name= azureconfig['deployment_name']

    return deployment_name

    

In [ ]:
def call(deployment_name, article, name):
    #Right now calling requests to get the synopsis, once Dataset is ready, replace it
    #context=get_synopsis(series, episode, context)

    #OpenAI Stuff
     system_message={"role": "system", "content": "You are a helpful assistant."}
     max_response_tokens = 4096
     token_limit = 2000
     conversation = []
     conversation.append(system_message)

     #print(len(article))
     if name=='cnn_dailymail':
         user_input = prompt_CNN(article)

     else:
         user_input = prompt(article)

         


   

    # print(f"Adword was {adword}")
    
     conversation.append({"role": "user", "content": user_input})

    #conv_history_tokens = num_tokens_from_messages(conversation)
    # print(conv_history_tokens)

    # while conv_history_tokens + max_response_tokens >= token_limit:
    #     del conversation[0] 
    #     conv_history_tokens = num_tokens_from_messages(conversation)

     try:
        response = openai.ChatCompletion.create(
        engine=deployment_name, # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
        messages=conversation,
        temperature=0.7,
        max_tokens=max_response_tokens,
    )
     except Exception as e:
        #  print('In Exception')
        #  print(e)
         return []
     
         

    # print(response['choices'][0]['message']['content'])

     try:   
        conversation.append({"role": "assistant", "content": response['choices'][0]['message']['content']})
     except Exception as e:
        #  print('In Exception')
        #  print(e)
         return []
         

    #print(conversation)

     responses=prompt_output_parse(response['choices'][0]['message']['content'])

     #print(responses)

    # final_res.append(responses)

    # if not responses:
    #     #return empty
    #     return None


     return responses
        
        
            





In [ ]:
dep_name=deployment_stuff()
# print(dep_name)

# print(series_title[0])
# print(episode_title[0])
dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'
name='cnn_dailymail'
dataset=dataset['test']



res=[]
for article in tqdm(dataset[article_key]):
    res.append(call(deployment_name, article, name))

with open('cnn.pkl', 'wb') as f:
  
    pkl.dump(res,f)


dataset = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'
dataset=dataset['test']


name='xsum'



res=[]
for article in tqdm(dataset[article_key]):
 
    res.append(call(deployment_name, article, name))

with open('xsum.pkl', 'wb') as f:
    pkl.dump(res,f)

dataset = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
dataset = DatasetDict({
    'train': dataset['test'],
        'test': dataset['train']})
dataset=dataset['test']

name='argilla/news-summary'


res=[]
for article in tqdm(dataset[article_key]):
 
    res.append(call(deployment_name, article, name))

with open('news.pkl', 'wb') as f:
    pkl.dump(res,f)



dataset = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
# 80% train, 20% test + validation
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


dataset=dataset['test']

name='reddit_tifu'

res=[]
for article in tqdm(dataset[article_key]):
  
    res.append(call(deployment_name, article, name))


with open('reddit.pkl', 'wb') as f:
    pkl.dump(res,f)

# with open("alternate_names.pkl", 'wb') as f:
#     pkl.dump(final_res,f)


    

    
    

    

In [163]:
with open('reddit_new_new.pkl', 'rb') as f:
  
    res=pkl.load(f)

In [164]:
len(res)

4214

In [165]:
count=0
for re in res:
    if re:
        count+=1
print(count)


3563


In [ ]:
res[0]

In [ ]:
dep_name=deployment_stuff()
# print(dep_name)

# print(series_title[0])
# print(episode_title[0])



dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'
name='cnn_dailymail'
dataset=dataset['test']

with open('cnn.pkl', 'rb') as f:
    prev=pkl.load(f)

res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))

with open('cnn_new.pkl', 'wb') as f:
  
    pkl.dump(res,f)


dataset = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'
dataset=dataset['test']


name='xsum'

with open('xsum.pkl', 'rb') as f:
    prev=pkl.load(f)

res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))
 

with open('xsum_new.pkl', 'wb') as f:
    pkl.dump(res,f)



dataset = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
dataset = DatasetDict({
    'train': dataset['test'],
        'test': dataset['train']})
dataset=dataset['test']

name='argilla/news-summary'

with open('news.pkl', 'rb') as f:
    prev=pkl.load(f)


res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))

with open('news_new.pkl', 'wb') as f:
    pkl.dump(res,f)



dataset = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
# 80% train, 20% test + validation
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


dataset=dataset['test']

name='reddit_tifu'


with open('reddit.pkl', 'wb') as f:
    pkl.dump(res,f)

res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))

with open('reddit_new.pkl', 'wb') as f:
    pkl.dump(res,f)

# with open("alternate_names.pkl", 'wb') as f:
#     pkl.dump(final_res,f)


    

    
    

    

In [144]:
dep_name=deployment_stuff()



dataset = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
# 80% train, 20% test + validation
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


dataset=dataset['test']

name='reddit_tifu'

res=[]
for article in tqdm(dataset[article_key]):
    sleep(1)
    res.append(call(deployment_name, article, name))

with open('reddit_new.pkl', 'wb') as f:
    pkl.dump(res,f)


dataset = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
dataset = DatasetDict({
    'train': dataset['test'],
        'test': dataset['train']})
dataset=dataset['test']

name='argilla/news-summary'

with open('news_new.pkl', 'rb') as f:
    prev=pkl.load(f)


res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))
        sleep(1)

with open('news_new_new.pkl', 'wb') as f:
    pkl.dump(res,f)



dataset = load_dataset("xsum")
article_key = 'document'
summary_key = 'summary'
dataset=dataset['test']


name='xsum'

with open('xsum_new.pkl', 'rb') as f:
    prev=pkl.load(f)

res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    sleep(1)
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))
        sleep(1)
 

with open('xsum_new_new.pkl', 'wb') as f:
    pkl.dump(res,f)


dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'
name='cnn_dailymail'
dataset=dataset['test']

with open('cnn_new.pkl', 'rb') as f:
    prev=pkl.load(f)

res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))
        sleep(1)

with open('cnn_new_new.pkl', 'wb') as f:
  
    pkl.dump(res,f)




100%|██████████| 4214/4214 [13:47:23<00:00, 11.78s/it]      
1000it [17:46,  1.07s/it]
11334it [20:51:13,  6.62s/it]  
11490it [3:27:11,  1.08s/it]


In [149]:
with open('cnn_new_new.pkl', 'wb') as f:
  
    pkl.dump(res,f)

In [162]:
dep_name=deployment_stuff()

dataset = load_dataset('reddit_tifu', 'long')


article_key = 'documents'
summary_key = 'tldr'
# 80% train, 20% test + validation
train_testvalid = dataset['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})


dataset=dataset['test']

name='reddit_tifu'

with open('reddit_new.pkl', 'rb') as f:
    prev=pkl.load(f)


res=[]
for article, pre in tqdm(zip(dataset[article_key],prev)):
    if pre:
        res.append(pre)
    else:
        res.append(call(deployment_name, article, name))
        sleep(1)

with open('reddit_new_new.pkl', 'wb') as f:
  
    pkl.dump(res,f)


4214it [28:55,  2.43it/s]
